# Convert the Neurominidase input file from Narupa 1 to narupa-openmm

The current version of Narupa comes with a XML format to describe simulations. We want to be able to run the Tamiflu-Neuraminidase demo with the new `narupa.openmm` server, but the XML format is not supported. Here, we read the XML file distributed with Narupa-server and generate an XML file for narupa-openmm.

We read the file `old/Tamiflu_Neuraminidase.xml` and `old/AO_no_water.pdb`, and writes `neuraminidase.xml`.

In [1]:
INPUT_PATH = 'old/Tamiflu_Neuraminidase.xml'
PDB_PATH = 'old/AO_no_water.pdb'
OUTPUT_PATH = 'neuraminidase.xml'

In [2]:
import simtk.openmm as mm
from simtk.openmm import app
from simtk.unit import picosecond, kelvin

from narupa.openmm.serializer import serialize_simulation

In [3]:
import numpy as np

In [4]:
from xml.dom.minidom import parse

In [5]:
document = parse(INPUT_PATH)

In [6]:
box_node = document.getElementsByTagName('SimulationBoundary')[0]
box_string = box_node.getAttribute('SimulationBox')
box_diagonal = box_string.split(',')

In [7]:
new_box_node = document.createElement('PeriodicBoxVectors')
for length, tag_name, attribute_name in zip(box_diagonal, 'ABC', 'xyz'):
    node = document.createElement(tag_name)
    for axis in 'xyz':
        node.setAttribute(axis, '0')
    node.setAttribute(attribute_name, length)
    new_box_node.appendChild(node)

In [8]:
print(new_box_node.toprettyxml())

<PeriodicBoxVectors>
	<A x="20" y="0" z="0"/>
	<B x="0" y="20" z="0"/>
	<C x="0" y="0" z="20"/>
</PeriodicBoxVectors>



In [9]:
system_node = document.getElementsByTagName('OpenMMForceField')[0]

In [10]:
system_node.tagName = 'System'

In [11]:
system_node.setAttribute('openmmVersion', '7.2.2')
system_node.setAttribute('type', 'System')
system_node.setAttribute('version', '1')
system_node.appendChild(new_box_node)

<DOM Element: PeriodicBoxVectors at 0x7fd3081ed508>

In [12]:
system_xml_string = system_node.toprettyxml()

In [13]:
system = mm.XmlSerializer.deserialize(system_xml_string)

In [14]:
thermostat = mm.AndersenThermostat(300 * kelvin, 0.01)

In [15]:
system.addForce(thermostat)

6

In [16]:
integrator_node = document.getElementsByTagName('Integrator')[0]
integrator_string = integrator_node.toprettyxml()
print(integrator_string)

<Integrator TimeStep="0.0005" Type="Verlet"/>



In [17]:
integrator = mm.VerletIntegrator(0.0005 * picosecond)

In [18]:
pdb = app.PDBFile(PDB_PATH)

In [19]:
simulation = app.Simulation(pdb.topology, system, integrator)

In [20]:
simulation.context.setPositions(pdb.positions)

In [21]:
simulation.step(10)

In [22]:
with open(OUTPUT_PATH, 'w') as outfile:
    outfile.write(serialize_simulation(simulation))